In [1]:
import sqlite3 as db 
import pandas as pd
conn = db.connect('soccer/database.sqlite')

In [2]:
print(conn)

In [3]:


all_players_info = pd.read_sql_query("""select  p.player_name,pa2.* from player_attributes pa2, player p where p.player_api_id = pa2.player_api_id and
(pa2.player_api_id, pa2.date) in (
select z.player_api_id, z.date from (
select pa.player_api_id, pa.date, min(abs(strftime('%s',pa.date) - strftime('%s','2014-09-01 00:00:00'))) from player_attributes pa, player p 
                                where pa.player_api_id = p.player_api_id
                                and strftime('%s',pa.date) > strftime('%s','2014-05-14 00:00:00') 
                                and pa.date like '2014 %'
                                group by pa.player_api_id ) z )""",conn)


In [109]:

year_data = {'2014': 'statbunker-football-stats/Player Stats 2014-15.csv' , 
             '2015': 'statbunker-football-stats/Player Stats 2015-16.csv' ,
             '2016': 'statbunker-football-stats/Player Stats 2016-17.csv'}

#ptm = pd.read_csv('statbunker-football-stats/Player Stats 2014-15.csv', )


# #caller.set_index('key').join(other.set_index('key'))
# joined = all_players_info.set_index('player_name').join(ptm.set_index('Player'), how='inner')



def agg_attributes(team):
    #team[team['Position'] == 'Goalkeeper']['Position'] = 'Defender'
    
    team.loc[lambda df: df['Position'] == 'Goalkeeper', 'Position'] = 'Defender'
       
    print(team['overall_rating'])
    g = team.groupby(['Position'] , as_index=False)
    print(g.mean())

def create_team( name, year):
    
    ptm = pd.read_csv(year_data[year] , dtype = {'overall_rating':float} )
   
    ptm = ptm[(ptm['Type Of Goal'] == 'Player') & (ptm['League'] == 'Premier League')]
    joined = all_players_info.set_index('player_name').join(ptm.set_index('Player'), how='inner')
    full_team = joined[joined['Team'] == name]
        
    pos_sel = {'Defender': 4, 'Midfielder': 3, 'Forward': 3, 'Goalkeeper':1}
    team_agg = {}
    
    team_df = {}
    
    df_start = full_team[full_team['Team'] == 'THIS DOES NOT EXIST']
    
    for key,value in pos_sel.items():
        
        grp = full_team[full_team['Position'] == key].sort_values(by = ['overall_rating'], ascending = False).head(value)
        df_start = df_start.append(grp)
        avg = grp.mean()
        team_agg[key] = avg
    
    return df_start
    
start11 = create_team('Everton', '2014')

agg_attributes(start11)


Leighton Baines    82.0
Seamus Coleman     79.0
Antolin Alcaraz    74.0
John Stones        73.0
Gareth Barry       80.0
Aaron Lennon       79.0
James McCarthy     79.0
Samuel Eto'o       81.0
Kevin Mirallas     80.0
Romelu Lukaku      80.0
Tim Howard         82.0
Name: overall_rating, dtype: float64
     Position             id  player_fifa_api_id  player_api_id  \
0    Defender  106162.200000       140524.800000   81878.000000   
1     Forward  135408.333333       124785.333333   80355.333333   
2  Midfielder   45849.333333       115942.000000   51956.333333   

   overall_rating  potential   crossing  finishing  heading_accuracy  \
0       78.000000  80.800000  59.200000  45.600000         61.800000   
1       80.333333  82.666667  73.000000  81.000000         72.666667   
2       79.333333  80.666667  75.666667  64.333333         54.666667   

   short_passing          ...            Min/goal  Assists  % Assists  \
0           66.2          ...                 NaN      9.0       20.